<table style="margin: 0; text-align: left; background-color: #050A0E; border: 1px solid lightblue;">
    <tr>
        <td style="direction: rtl; text-align: right; color: #FFF8E7; ">
            <h2 style="color:#FFF8E7;">JobTonic</h2>
            <span style="color:#FFF8E7; font-size: 12px">
                כי חיפוש עבודה ראוי להרגיש כמו התחלה חדשה, לא כמו מסע מייגע.
                במקום להיסחף בגלים של מידע, 
                תנו לנו להיות הרוח שמכוונת את המפרש.
                הכלי
                 מזקק עבורכם את מהות כל משרה — בבהירות, במהירות, ובדיוק שמחזיר לכם שליטה.
                זה לא רק לחפש עבודה. זה לדייק מטרה. זה להתקדם עם ראש צלול, לב פתוח, וביטחון אמיתי.
                בכל שלב בדרך – אנחנו האנרגיה שמזיזה אתכם קדימה.
            </span>
        </td>
<td style="width: 290px; height: 200px; vertical-align: middle;">
            <img src="JobTonic.png" style="width: 100%; height: 100%; display: block;" />
        </td>
    </tr>
</table>

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import time
import json
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

In [21]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [22]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with Selenium for dynamic websites.
    """

    def __init__(self, url):
        self.url = url

        # Set up Selenium WebDriver with Chrome
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # Run in headless mode
        options.add_argument('--disable-gpu')
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

        # Load the webpage
        driver.get(url)
        time.sleep(5)  # Wait for the page to load completely

        # Get the page source and parse it with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()  # Close the browser

        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""

        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [35]:
# The system prompt for the careers page
find_careers_system_prompt = "You are provided with a list of links found \
on a webpage. You are able to find the link that is most likely to be a Careers page."
find_careers_system_prompt += "You should respond a single link in the following format: \
    https://another.full.url/careers"
find_careers_system_prompt += "\nIn the case that there are no relevant link, respond with an Null value: null"

# The system prompt for jobs links
job_links_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to a keyword that was provided, \
\n"
job_links_system_prompt += "You should respond in JSON as in this example:"
job_links_system_prompt += """
{
    "links": [
        {"job title": "Software Engineer", "url": "https://full.url/goes/here/software-engineer"},
        {"job title": "Data Scientist", "url": "https://full.url/goes/here/data-scientist"},
        {"job title": "Product Manager", "url": "https://full.url/goes/here/product-manager"},
        {"job title": "Data Analyst", "url": "https://full.url/goes/here/data-analyst"},
        {"job title": "Web Developer", "url": "https://full.url/goes/here/web-developer"}
    ]
}
"""
job_links_system_prompt += "\nIn the case that there are no relevant links, respond with an empty JSON object: {}"

# The system prompt for job postings
job_posting_system_prompt = (
    "You are an assistant specialized in summarizing job postings.\n"
    "Your task is to extract and clearly summarize three things from the input job description:\n"
    "1. How the company describes itself.\n"
    "2. What the day-to-day responsibilities of the role are.\n"
    "3. What the qualifications or requirements are to apply.\n\n"
    "Write the summary in plain, professional English. Keep each section concise (1–2 sentences).\n"
    "Do not copy text verbatim unless necessary. Structure your response under the following headers:\n"
    "- About the Company\n"
    "- Role Responsibilities\n"
    "- Qualifications and Requirements\n"
    "in addition to the following fields:\n"
    "- Company Name\n"
    "- Company URL\n"
    "- Company Location\n"
    "- Company Size\n"
    "- Company Website\n"
    "- Company Founded\n"
    "- Company Mission\n"
    "- Company Values\n"
)
job_posting_system_prompt += "\nYou should respond in JSON as in this example:"
job_posting_system_prompt += """
{
    "company_name": "Tech Innovations Inc.",
    "job_url": "https://full.url/goes/here/software-engineer",
    "company_location": "San Francisco, CA",
    "company_size": "500+ employees",
    "company_website": "https://www.techinnovations.com",
    "company_founded": "2010",
    "company_mission": "To innovate and simplify technology for everyone.",
    "company_values": "Innovation, Integrity, Customer Focus",
    "company_description": "The company is a leading provider of innovative technology solutions.",
    "role_responsibilities": "The role involves developing software applications and collaborating with cross-functional teams.",
    "qualifications_requirements": "Candidates should have a degree in Computer Science and experience with Python."
}
"""
job_posting_system_prompt += "\nIn the case that there are no relevant links, respond with an empty JSON object: {}"

In [14]:
print(find_careers_system_prompt)

You are provided with a list of links found on a webpage. You are able to find the link that is most likely to be a Careers page.You should respond a single link in the following format:     https://another.full.url/careers


In [ ]:
print(job_links_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to a keyword that was provided, 
You should respond in JSON as in this example:
{
    "links": [
        {"job title": "Software Engineer", "url": "https://full.url/goes/here/software-engineer"},
        {"job title": "Data Scientist", "url": "https://full.url/goes/here/data-scientist"},
        {"job title": "Product Manager", "url": "https://full.url/goes/here/product-manager"},
        {"job title": "Data Analyst", "url": "https://full.url/goes/here/data-analyst"},
        {"job title": "Web Developer", "url": "https://full.url/goes/here/web-developer"}
    ]
}

In the case that there are no relevant links, respond with an empty JSON object: {}


In [17]:
print(job_posting_system_prompt)

You are an assistant specialized in summarizing job postings.
Your task is to extract and clearly summarize three things from the input job description:
1. How the company describes itself.
2. What the day-to-day responsibilities of the role are.
3. What the qualifications or requirements are to apply.

Write the summary in plain, professional English. Keep each section concise (1–2 sentences).
Do not copy text verbatim unless necessary. Structure your response under the following headers:
- About the Company
- Role Responsibilities
- Qualifications and Requirements
in addition to the following fields:
- Company Name
- Company URL
- Company Location
- Company Size
- Company Website
- Company Founded
- Company Mission
- Company Values

You should respond in JSON as in this example:
{
    "company_name": "Tech Innovations Inc.",
    "job_url": "https://full.url/goes/here/software-engineer",
    "company_location": "San Francisco, CA",
    "company_size": "500+ employees",
    "company_we

In [48]:
# This function is used to create a user prompt to find the careers page
def get_careers_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these is the Careers page, \
        respond with the full https URL." 
    user_prompt += "\nLinks (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

# This function is used to create a user prompt to find the job links
def get_job_links_user_prompt(website, keyword):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += f"please decide which of these are relevant job links for the keyword '{keyword}', \
        respond with the full https URL in JSON format.\n" 
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

# This function is used to create a user prompt to find the job postings
def get_job_postings_user_prompt(website):
    user_prompt = "Please analyze the following job description:\n\n"
    user_prompt += website.text
    return user_prompt


In [49]:
# Function

# This function find the career page link of a company 
# based on url
def find_career_page(company_url):
    website = Website(company_url)
    # Use OpenAI API to find the career page link
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": find_careers_system_prompt},
            {"role": "user", "content": get_careers_user_prompt(website)}
        ],
        # headers=HEADERS
    )
    # Parse the response
    response_text = response.choices[0].message.content
    # Extract the career page link from the response
    career_page_link = response_text.strip()
    if career_page_link.startswith("http"):
        return career_page_link
    else:
        # If the response is not a valid URL, return None
        return None

# This function find all relevant job postings links 
# on the career page and return the relevant 
# job postings links by a keyword
def find_job_postings(career_page_url, keyword):
    website = Website(career_page_url)
    print(website.links)
    # Use OpenAI API to find the job postings links
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": job_links_system_prompt},
            {"role": "user", "content": get_job_links_user_prompt(website, keyword)}
        ],
        # headers=HEADERS,
        response_format={"type": "json_object"}
    )
    # Parse the response
    response_text = response.choices[0].message.content
    # Extract the job postings links from the response
    job_postings_links = response_text.strip()
    return job_postings_links

# This function summarize each job posting from the list and save it to a file
def job_postings(job_postings_url):
    website = Website(job_postings_url)
    # Use OpenAI API to summarize the job posting
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": job_posting_system_prompt},
            {"role": "user", "content": get_job_postings_user_prompt(website)}
        ],
        # headers=HEADERS,
        response_format={"type": "json_object"}
    )
    # Parse the response
    response_text = response.choices[0].message.content
    # Extract the job posting summary from the response
    job_posting_summary = response_text.strip()
    print(job_posting_summary)


In [50]:
# This function wraps the three functions above by taking a company url and a keyword and return a summary of each job posting
def print_career_page(company_url):
    career_page_url = find_career_page(company_url)
    if career_page_url:
        print(f"Career page URL: {career_page_url}")
    else:
        print("No career page found.")

In [46]:
#company_url = "https://www.microsoft.com/en-il/"
# company_url = "https://www.google.com/"
# company_url = "https://www.ibm.com/"
company_url = "https://hellosimply.com/"
print_career_page(company_url)

Career page URL: https://hellosimply.com/about/careers


In [52]:
def find_job_postings_by_keyword(url, keyword):     
    job_postings_urls = find_job_postings(url, keyword)
    if job_postings_urls:
        print(f"Job postings URLs: {job_postings_urls}")
        # for job_posting_url in job_postings_urls:
        #     job_postings(job_posting_url)
    else:
        print("No relevant job postings found.")

In [53]:
# Example usage
#job_search_url = "https://jobs.careers.microsoft.com/global/en/search?q=AI&lc=Israel&l=en_us&pg=1&pgSz=20&o=Relevance&flt=true&ulcs=true&cc=Israel&ref=cms"
job_search_url = "https://hellosimply.com/careers"
keyword = "AI engineer"

# Find job postings by keyword
find_job_postings_by_keyword(job_search_url, keyword)

['/', '/about', '/careers', 'https://piano-help.hellosimply.com', 'https://join-piano.hellosimply.com/router?utm_source=si_website&utm_campaign=si_website_header', '/simply-piano', '/simply-guitar', '/simply-sing', '/simply-tune', '/simply-piano-xr', '/simply-draw', '/about', '/careers', 'https://piano-help.hellosimply.com', 'https://www.facebook.com/hellosimply', 'https://www.instagram.com/hellosimply', 'https://x.com/hellosimply', 'https://www.youtube.com/c/hellosimply', 'https://www.linkedin.com/company/simply-joytunes/', 'https://www.youtube.com/c/hellosimply', 'https://www.linkedin.com/company/simply-joytunes/', 'https://medium.com/hellosimply', 'https://www.galiblochliran.com/podcast/on-culture-co-builders-and-founding-a-startup-with-your-own-brother-with-yuval-kaminka', 'https://www.youtube.com/watch?v=qm74rg-LT9Q', 'https://www.youtube.com/watch?v=witKIw0XxgY', 'https://medium.com/hellosimply/from-zero-to-one-the-story-of-simply-guitar-c6ae21c21a66', 'https://www.youtube.com/wa

***Second Try***

In [54]:
def classify_job_source(url: str) -> dict:
    prompt = f"""
You are a web analyst AI. I will give you the URL of a careers or jobs page from a company's website.

Your task is to infer **how the job listings are loaded** on the page. Based on your prior knowledge (without browsing the web), determine which of the following methods best matches the site's behavior:

1. The jobs are loaded via a third-party platform such as Greenhouse, Workday, Lever, or similar.
2. The site uses an internal or external **API** (public or private) to load job data dynamically.
3. The jobs are rendered as plain static **HTML**, directly embedded in the page.

---

Return your analysis in this structured JSON format:

{{
  "url": "{url}",
  "likely_method": "Greenhouse",  // or "API", "Workday", "Static HTML", "Unknown"
  "explanation": "Explain why you believe this is the correct classification, based on patterns in the URL, structure, or known platform usage."
}}
"""
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        response_format={"type": "json_object"}
    )
    return response.choices[0].message.content
# Example usage
url = "https://hellosimply.com/careers"
result = classify_job_source(url)
print(result)

{  
  "url": "https://hellosimply.com/careers",
  "likely_method": "Workday",  
  "explanation": "Based on the URL pattern and the company's website structure, I believe that the job listings are loaded via Workday. The presence of '/careers' in the URL suggests a dedicated careers page, which is common for companies using Workday to manage their job postings. Additionally, Workday's platform is known for its integration with company websites, allowing for seamless management of job listings and applicant tracking. This classification is based on patterns observed in other Workday-powered career pages."  
}


In [60]:
classification = result
if isinstance(classification, str):
    try:
        classification = json.loads(classification)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON string provided for 'classification'.")

In [61]:
def extract_jobs_from_classification(classification: dict) -> dict:
    method = classification["likely_method"]
    url = classification["url"]

    prompt = f"""
You previously determined that the job listings on the following site are loaded via: {method}.

Now, based on that classification, extract a structured list of job postings from this site:

URL: {url}

---

Return the result as a JSON array of job objects, each with the following fields:

- "title": the name of the position
- "location": the city and/or country (if available)
- "url": a direct link to the job posting

Example format:

[
  {{
    "title": "Senior Frontend Engineer",
    "location": "Berlin, Germany",
    "url": "https://example.com/careers/frontend-engineer"
  }},
  ...
]

If you cannot find any jobs or the structure is unclear, return an empty array with a short explanation like this:

{{
  "jobs": [],
  "note": "The page structure appears dynamic or requires JavaScript execution to access job data."
}}
"""
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        response_format={"type": "json_object"}
    )
    return response.choices[0].message.content

# Example usage
result = extract_jobs_from_classification(classification)
print(result)


{

  "jobs": [
    {
      "title": "",
      "location": "",
      "url": ""
    },
    {
      "title": "",
      "location": "",
      "url": ""
    }
  ],
  "note": "The page structure appears dynamic or requires JavaScript execution to access job data."
}
